In [2]:
from eptnr.rewards.egalitarian import EgalitarianTheilReward, BaseReward
from eptnr.constants.travel_metric import TravelMetric

from eptnr.analysis.full_problem_exploration import full_problem_exploration
from eptnr.plotting.solution_plotting import plot_full_problem_exploration
from inequality.theil import TheilD
import igraph as ig
import numpy as np
import geopandas as gpd
import pandas as pd

In [3]:
graph = ig.read('./test_cost_data/graph.picklez')
gdf = gpd.read_file('./test_cost_data/census_data.geojson')

In [2]:
from typing import List
from eptnr.rewards.egalitarian import InverseTheilReward

class InverseTheilAndCostReward(BaseReward):

    def __init__(self, census_data: pd.DataFrame, com_threshold: float, monetary_budget: float,
                 groups: List[str] = None, metrics: List[TravelMetric] = None,
                 verbose: bool = False):
        super().__init__(census_data, com_threshold, groups, metrics, verbose)
        self.monetary_budget = monetary_budget
        self.inverse_theil = InverseTheilReward(census_data, com_threshold, groups, metrics, verbose)

    def _weight_calculation(self, min_rewards, max_rewards, ):
        pass

    def _evaluate(self, g: ig.Graph, *args, **kwargs) -> float:
        """

        Args:
            g:

        Returns:

        """
        inverse_theil_reward = self.inverse_theil.evaluate(g, *args, **kwargs)

        removed_edges = g.es.select(active=0)
        total_savings = sum([e['cost'] for e in removed_edges])

        weighted_equality_reward = self._weight_calculation() * inverse_theil_reward
        weighted_price_reward = self._weight_calculation() * abs(total_savings - self.monetary_budget)
        total_reward = weighted_equality_reward +

        return None

    def _reward_scaling(self, reward: float) -> float:
        return np.log(len(self.census_data)) - reward